In [20]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb, perm
from collections import defaultdict
import string


def print_mat(mat):
    for row in mat:
        print(''.join(row))


files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [26]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))

P = list(map(lambda l: 1 if l == '>' else -1, lines[0]))

R = [
    [(0, 0), (1, 0), (2, 0), (3, 0)],
    [(1, 0), (1, 1), (1, 2), (0, 1), (2, 1)],
    [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],
    [(0, 0), (0, 1), (0, 2), (0, 3)],
    [(0, 0), (0, 1), (1, 0), (1, 1)],
]

print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [27]:
def simulate(iter):
    T = np.matrix([[False] * 7] * 20)
    V = set()
    maxy = -1
    i = 0
    pi = 0
    ri = 0
    V = set()
    I = {}
    Y = defaultdict(lambda: 0)
    skip = 0
    while i < iter:
        state = (pi, ri, str(T[maxy-64:maxy+3, :].tolist()))
        if skip == 0:
            if state in V:
                interval = i - I[state]
                increment = maxy - Y[state]
                n = (iter - i) // interval
                i += interval * n
                skip += increment * n
            else:
                I[state] = i
                Y[state] = maxy
                V.add(state)

        r = [(x + 2, y + maxy + 4)for x, y in R[ri]]
        ri = (ri + 1) % 5

        while True:
            # Jet:
            dx = P[pi]
            pi = (pi + 1) % len(P)
            rnew = []
            for x, y in r:
                if x + dx not in range(0, 7) or T[y, x+dx]:
                    break
                rnew.append((x + dx, y))
            if len(rnew) == len(r):
                r = rnew

            # Fall:
            rnew = []
            for x, y in r:
                if y - 1 < 0 or T[y-1, x]:
                    break
                rnew.append((x, y - 1))
            if len(rnew) == len(r):
                r = rnew
            else:
                break

        for x, y in r:
            T[y, x] = True
            maxy = max(maxy, y)

        if T.shape[0] < maxy + 8:
            T = np.vstack((T, np.matrix([[False] * 7] * 7)))

        i += 1

    return maxy + 1 + skip


simulate(2022)


3179

### Part 2

In [28]:
simulate(1000000000000)

1567723342929